# Preparação de dados

## Dados utilizados

O conjunto de dados utilizado como base foi o [Breast Cancer Wisconsin (Diagnostic)](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic) ele é amplamente utilizado para análise e classificação de câncer de mama. Ele contém medições computacionais de características extraídas de imagens de massas celulares em exames de mamografia. As características foram calculadas a partir de imagens digitalizadas de aspirações por agulha fina (FNA), sendo usadas para prever se uma massa é maligna (câncer) ou benigna (não cancerígena). Estes dados foram disponibilizados no repositório [UCI Machine Learning](https://archive.ics.uci.edu/) e originalmente coletados pelo Dr. William H. Wolberg da University of Wisconsin Hospitals, Madison.

> Dados anotados **diretamente** por corpo médico regularmente registrado.

É preciso ter em mente que modelos de aprendizado de máquina implementados na área de saúde **sempre** devem:

- Ter sua acurácia acompanhada por retorno humano (Human Feedback)

- Ter explicabilidade (Explainability)

- Ter critérios de honestidade definidos (Fairness)

- (Para dados de Brasileiros) Obedecer a LGPD para treinamento do modelo e inferência dos dados

- (Na União Européia) Obedecer a GPDR para treinamento do modelo e inferência dos dados

- (Nos Estados Unidos) Obedecer a HIPAA para treinamento do modelo e inferência dos dados

## Importar bibliotecas

In [ ]:
import os
import requests
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
from faker import Faker
import random

## Baixar dados

In [ ]:
# Link para o dataset
url = "https://archive.ics.uci.edu/static/public/17/breast+cancer+wisconsin+diagnostic.zip"

# Pasta de destino onde o arquivo será descompactado (na raiz do projeto)
dest_folder = "../data/breast_cancer"

# Arquivo zip de destino
dest_zip_file = os.path.join(dest_folder, "breast_cancer.zip")

# Arquivo parquet de destino
dest_parquet_file = os.path.join(dest_folder, "breast_cancer.parquet")

# Arquivo parquet de destino
dest_parquet_laudos_file = os.path.join(dest_folder, "breast_cancer_laudos.parquet")

if not os.path.exists(dest_folder):

    # Crie a pasta se ela não existir
    print(f"Criando a pasta {dest_folder}...")
    os.makedirs(dest_folder, exist_ok=True)

    # Faça o download e salve o arquivo
    print("Baixando o dataset...")
    response = requests.get(url)
    response.raise_for_status()  # Levanta um erro se o download falhar

    with open(dest_zip_file, "wb") as f:
        f.write(response.content)
    print(f"Download concluído e salvo em {dest_zip_file}")

    # Descompacte o arquivo
    print("Descompactando o arquivo...")
    with zipfile.ZipFile(dest_zip_file, 'r') as zip_ref:
        zip_ref.extractall(dest_folder)

    print(f"Arquivos descompactados em {dest_folder}")
else:
    print(f"Os arquivos já existem em {dest_folder}")

## Ajustar colunas

In [ ]:
# Caminho para os arquivos de dados e nomes
data_file = os.path.join(dest_folder, "wdbc.data")

# Carregar os dados
data = pd.read_csv(data_file, header=None, sep=",")

# 12 primeiras colunas
data = data.iloc[:, :12]

# Definir os nomes das colunas
column_names = [
    "id_number",
    "diagnosis",
    "radius",
    "texture",
    "perimeter",
    "area",
    "smoothness",
    "compactness",
    "concavity",
    "concave points",
    "symmetry",
    "fractal dimension"
]

# Definir os nomes das colunas no DataFrame
data.columns = column_names

# Adicionar coluna diagnosis_y com valores 0 e 1
data['diagnosis_y'] = data['diagnosis'].map({'M': 1, 'B': 0})

# Exibir as primeiras linhas do DataFrame
data.head()

## Salvar dados

In [ ]:
# salvar dataframe em um arquivo parquet na pasta data
data.to_parquet(dest_parquet_file)

# Análise exploratória dos dados

## Formato dos dados

In [ ]:
data.shape

## Mostrar informações

In [ ]:
data.info()

## Estatísticas descritivas

In [ ]:
data.describe()

## Verificar valores ausentes

In [ ]:
data.isnull().sum()

## Balanceamento das classes

In [ ]:
data["diagnosis"].value_counts()

## Distribuição por diagnóstico

In [ ]:
# Plotar a distribuição dos atributos por diagnóstico
sns.pairplot(data, hue='diagnosis', vars=['radius', 'texture', 'perimeter', 'area'], palette='Set2')
plt.suptitle("Distribuição dos Atributos por Diagnóstico", y=1.02)
plt.show()

## Correlação entre features

In [ ]:
numeric_cols = ['radius', 'texture', 'perimeter', 'area', 'smoothness',
                'compactness', 'concavity', 'concave points', 'symmetry', 'fractal dimension', 'diagnosis_y']

correlation_matrix = data.drop(['id_number', 'diagnosis'], axis=1).corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlação entre Features')
plt.tight_layout()
plt.show()

## Distribuição das features

In [ ]:
numeric_cols = ['radius', 'texture', 'perimeter', 'area', 'smoothness',
                'compactness', 'concavity', 'concave points', 'symmetry', 'fractal dimension']

fig, axes = plt.subplots(5, 2, figsize=(15, 20))
axes = axes.ravel()

for idx, col in enumerate(numeric_cols):
    axes[idx].hist(data[data['diagnosis'] == 'M'][col], alpha=0.5, label='Maligno', bins=30, color='red')
    axes[idx].hist(data[data['diagnosis'] == 'B'][col], alpha=0.5, label='Benigno', bins=30, color='blue')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequência')
    axes[idx].legend()
    axes[idx].set_title(f'Distribuição de {col}')

plt.tight_layout()
plt.show()

## Boxplots para identificar outliers

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(15, 20))
axes = axes.ravel()

for idx, col in enumerate(numeric_cols):
    sns.boxplot(data=data, x='diagnosis', y=col, hue='diagnosis', ax=axes[idx], palette='Set2', legend=False)
    axes[idx].set_title(f'Boxplot de {col}')

plt.tight_layout()
plt.show()

# Extra: Dados sintéticos

## Gerar laudos baseado nos dados

In [ ]:
fake = Faker('pt_BR')

# Função para traduzir o diagnóstico
def traduzir_diagnostico(diagnostico):
    return "Benigno" if diagnostico == "B" else "Maligno"

# Função para gerar laudo
def gerar_laudo(row):
    paciente = fake.name()
    tamanho = round(row['radius'] * 2, 1)
    textura = "regular" if row['texture'] < 20 else "irregular"
    quadrante = random.choice(["superior esquerdo", "superior direito", "inferior esquerdo", "inferior direito"])
    diagnostico = traduzir_diagnostico(row['diagnosis'])

    return f"""
        Paciente: {paciente}
        Exame: Mamografia
        Data do Exame: {fake.date_this_year()}

        Descrição:
        Observou-se uma lesão de aproximadamente {tamanho} mm, localizada no quadrante {quadrante}, com bordas {textura}.
        O exame sugere que a lesão apresenta características {diagnostico.lower()}.

        Conclusão: {diagnostico}.
        Recomendação: {('Acompanhar evolução com novo exame em 6 meses' if diagnostico == 'Benigno' else 'Encaminhar para biópsia e avaliação oncológica')}.
        """

# Aplicar em todas as linhas do dataset
data['laudo'] = data.apply(gerar_laudo, axis=1)

## Verificar laudos gerados

In [ ]:
# Mostre apenas as colunas diagnosis e laudo
data[['id_number','diagnosis', 'laudo']].head()

In [ ]:
# Mostre o texto completo do laudo para os primeiros 3 diagnósticos malignos
for laudo in data[data['diagnosis'] == 'M']['laudo'].head(3):
    print(laudo)
    print("=" * 80)

In [ ]:
# Mostre o texto completo do laudo para os primeiros 3 diagnósticos benignos
for laudo in data[data['diagnosis'] == 'B']['laudo'].head(3):
    print(laudo)
    print("=" * 80)

## Adicionar Ruido

In [ ]:
# Introduzir ruído nos dados
def adicionar_ruido(row):

    # row['radius'] += random.uniform(-1, 1)  # Pequeno ajuste aleatório no raio
    # row['texture'] += random.uniform(-1, 1)  # Pequeno ajuste aleatório na textura

    if random.random() < 0.05:  # 5% de chance de alterar o diagnóstico
        row['diagnosis'] = "B" if row['diagnosis'] == "M" else "M"

    return row

data = data.apply(adicionar_ruido, axis=1)

## Salvar dados

In [ ]:
# salvar dataframe em um arquivo parquet na pasta data
data.to_parquet(dest_parquet_laudos_file)